# Grounding with Memory: Using Context to Resolve References

## Introduction

In this notebook, you'll learn about grounding - how agents use memory to understand references and maintain context across a conversation. When users say "that course" or "my advisor", the agent needs to know what they're referring to. The Agent Memory Server's extracted memories provide this grounding automatically.

### What You'll Learn

- What grounding is and why it matters
- How extracted memories provide grounding
- How to handle references to people, places, and things
- How memory enables natural conversation flow

### Prerequisites

- Completed Section 3 notebooks
- Redis 8 running locally
- Agent Memory Server running
- OpenAI API key set

## Concepts: Grounding

### What is Grounding?

**Grounding** is the process of connecting references in conversation to their actual meanings. When someone says:

- "Tell me more about **that course**" - Which course?
- "When does **she** teach?" - Who is "she"?
- "Is **it** available online?" - What is "it"?
- "What about **the other one**?" - Which one?

The agent needs to **ground** these references to specific entities mentioned earlier in the conversation.

### Grounding Without Memory (Bad)

```
User: I'm interested in machine learning.
Agent: Great! We have CS401: Machine Learning.

User: Tell me more about that course.
Agent: Which course are you asking about? ❌
```

### Grounding With Memory (Good)

```
User: I'm interested in machine learning.
Agent: Great! We have CS401: Machine Learning.
[Memory extracted: "Student interested in CS401"]

User: Tell me more about that course.
Agent: CS401 covers supervised learning, neural networks... ✅
[Memory grounds "that course" to CS401]
```

### How Agent Memory Server Provides Grounding

The Agent Memory Server automatically:
1. **Extracts entities** from conversations (courses, people, places)
2. **Stores them** in long-term memory with context
3. **Retrieves them** when similar references appear
4. **Provides context** to ground ambiguous references

### Types of References

**Pronouns:**
- "it", "that", "this", "those"
- "he", "she", "they"

**Descriptions:**
- "the ML class"
- "my advisor"
- "the main campus"

**Implicit references:**
- "What are the prerequisites?" (for what?)
- "When does it meet?" (what meets?)

## Setup

In [ ]:
import os
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from agent_memory_client import MemoryAPIClient as MemoryClient, MemoryClientConfig

# Initialize
student_id = "student_789"
session_id = "grounding_demo"

# Initialize memory client with proper config
import os
config = MemoryClientConfig(
    base_url=os.getenv("AGENT_MEMORY_URL", "http://localhost:8000"),
    default_namespace="redis_university"
)
memory_client = MemoryClient(config=config)

llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

print(f"✅ Setup complete for {student_id}")

## Hands-on: Grounding Through Conversation

### Example 1: Grounding Course References

Let's have a conversation where we refer to courses in different ways.

In [ ]:
async def chat_turn(user_message, conversation_history):
    """Helper function to process a conversation turn."""
    
    # Search long-term memory for context
    memories = await memory_client.search_long_term_memory(
        text=user_message,
        limit=5
    )
    
    # Build context from memories
    memory_context = "\n".join([f"- {m.text}" for m in memories]) if memories else "None".memories
    
    system_prompt = f"""You are a helpful class scheduling agent for Redis University.

What you remember about this student:
{memory_context}

Use this context to understand references like "that course", "it", "the one I mentioned", etc.
"""
    
    # Build messages
    messages = [SystemMessage(content=system_prompt)]
    messages.extend(conversation_history)
    messages.append(HumanMessage(content=user_message))
    
    # Get response
    response = llm.invoke(messages)
    
    # Update conversation history
    conversation_history.append(HumanMessage(content=user_message))
    conversation_history.append(AIMessage(content=response.content))
    
    # Save to working memory (triggers extraction)
    messages_to_save = [
        {"role": "user" if isinstance(m, HumanMessage) else "assistant", "content": m.content}
        for m in conversation_history
    ]
    from agent_memory_client.models import WorkingMemory, MemoryMessage
    
    # Convert messages to MemoryMessage format
    memory_messages = [MemoryMessage(**msg) for msg in messages_to_save]
    
    # Create WorkingMemory object
    working_memory = WorkingMemory(
        session_id=session_id,
        user_id="demo_user",
        messages=memory_messages,
        memories=[],
        data={}
    )
    
    await memory_client.put_working_memory(
        session_id=session_id,
        memory=working_memory,
        user_id="demo_user",
        model_name="gpt-4o"
    )
    
    return response.content, conversation_history

print("✅ Helper function defined")

In [ ]:
# Start conversation
conversation = []

print("=" * 80)
print("CONVERSATION: Grounding Course References")
print("=" * 80)

# Turn 1: Mention a specific course
print("\n👤 User: I'm interested in CS401, the machine learning course.")
response, conversation = await chat_turn(
    "I'm interested in CS401, the machine learning course.",
    conversation
)
print(f"🤖 Agent: {response}")

# Wait for extraction
await asyncio.sleep(2)

# Turn 2: Use pronoun "it"
print("\n👤 User: What are the prerequisites for it?")
response, conversation = await chat_turn(
    "What are the prerequisites for it?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'it' to CS401")

# Turn 3: Use description "that ML class"
print("\n👤 User: Is that ML class available online?")
response, conversation = await chat_turn(
    "Is that ML class available online?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'that ML class' to CS401")

### Example 2: Grounding People References

Let's have a conversation about people (advisors, professors).

In [ ]:
# New conversation
conversation = []

print("\n" + "=" * 80)
print("CONVERSATION: Grounding People References")
print("=" * 80)

# Turn 1: Mention a person
print("\n👤 User: My advisor is Professor Smith from the CS department.")
response, conversation = await chat_turn(
    "My advisor is Professor Smith from the CS department.",
    conversation
)
print(f"🤖 Agent: {response}")

await asyncio.sleep(2)

# Turn 2: Use pronoun "she"
print("\n👤 User: What courses does she teach?")
response, conversation = await chat_turn(
    "What courses does she teach?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'she' to Professor Smith")

# Turn 3: Use description "my advisor"
print("\n👤 User: Can my advisor help me with course selection?")
response, conversation = await chat_turn(
    "Can my advisor help me with course selection?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'my advisor' to Professor Smith")

### Example 3: Grounding Place References

Let's talk about campus locations.

In [ ]:
# New conversation
conversation = []

print("\n" + "=" * 80)
print("CONVERSATION: Grounding Place References")
print("=" * 80)

# Turn 1: Mention a place
print("\n👤 User: I prefer taking classes at the downtown campus.")
response, conversation = await chat_turn(
    "I prefer taking classes at the downtown campus.",
    conversation
)
print(f"🤖 Agent: {response}")

await asyncio.sleep(2)

# Turn 2: Use pronoun "there"
print("\n👤 User: What CS courses are offered there?")
response, conversation = await chat_turn(
    "What CS courses are offered there?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'there' to downtown campus")

# Turn 3: Use description "that campus"
print("\n👤 User: How do I get to that campus?")
response, conversation = await chat_turn(
    "How do I get to that campus?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'that campus' to downtown campus")

### Example 4: Complex Multi-Reference Conversation

Let's have a longer conversation with multiple entities to ground.

In [ ]:
# New conversation
conversation = []

print("\n" + "=" * 80)
print("CONVERSATION: Complex Multi-Reference")
print("=" * 80)

# Turn 1
print("\n👤 User: I'm looking at CS401 and CS402. Which one should I take first?")
response, conversation = await chat_turn(
    "I'm looking at CS401 and CS402. Which one should I take first?",
    conversation
)
print(f"🤖 Agent: {response}")

await asyncio.sleep(2)

# Turn 2
print("\n👤 User: What about the other one? When is it offered?")
response, conversation = await chat_turn(
    "What about the other one? When is it offered?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'the other one' to the second course mentioned")

# Turn 3
print("\n👤 User: Can I take both in the same semester?")
response, conversation = await chat_turn(
    "Can I take both in the same semester?",
    conversation
)
print(f"🤖 Agent: {response}")
print("\n✅ Agent grounded 'both' to CS401 and CS402")

## Verify Extracted Memories

Let's check what memories were extracted to enable grounding.

In [ ]:
print("\n" + "=" * 80)
print("EXTRACTED MEMORIES (Enable Grounding)")
print("=" * 80)

# Get all memories
all_memories = await memory_client.search_long_term_memory(
    text="",
    limit=20
)

print("\nMemories that enable grounding:\n")
for i, memory in enumerate(all_memories.memories, 1):
    print(f"{i}. {memory.text}")
    print(f"   Type: {memory.memory_type} | Topics: {', '.join(memory.topics)}")
    print()

print("✅ These memories provide the context needed to ground references!")

## Key Takeaways

### How Grounding Works

1. **User mentions entity** (course, person, place)
2. **Agent Memory Server extracts** entity to long-term memory
3. **User makes reference** ("it", "that", "she", etc.)
4. **Semantic search retrieves** relevant memories
5. **Agent grounds reference** using memory context

### Types of Grounding

**Direct references:**
- "CS401" → Specific course
- "Professor Smith" → Specific person

**Pronoun references:**
- "it" → Last mentioned thing
- "she" → Last mentioned person
- "there" → Last mentioned place

**Description references:**
- "that ML class" → Course about ML
- "my advisor" → Student's advisor
- "the downtown campus" → Specific campus

**Implicit references:**
- "What are the prerequisites?" → For the course we're discussing
- "When does it meet?" → The course mentioned

### Why Memory-Based Grounding Works

✅ **Automatic** - No manual entity tracking needed
✅ **Semantic** - Understands similar references
✅ **Persistent** - Works across sessions
✅ **Contextual** - Uses conversation history
✅ **Natural** - Enables human-like conversation

### Best Practices

1. **Include memory context in system prompt** - Give LLM grounding information
2. **Search with user's query** - Find relevant entities
3. **Trust semantic search** - It finds related memories
4. **Let extraction happen** - Don't manually track entities
5. **Test with pronouns** - Verify grounding works

## Exercises

1. **Test ambiguous references**: Have a conversation mentioning multiple courses, then use "it". Does the agent ground correctly?

2. **Cross-session grounding**: Start a new session and refer to entities from a previous session. Does it work?

3. **Complex conversation**: Have a 10-turn conversation with multiple entities. Track how grounding evolves.

4. **Grounding failure**: Try to break grounding by using very ambiguous references. What happens?

## Summary

In this notebook, you learned:

- ✅ Grounding connects references to their actual meanings
- ✅ Agent Memory Server's extracted memories provide grounding automatically
- ✅ Semantic search retrieves relevant context for grounding
- ✅ Grounding enables natural, human-like conversations
- ✅ No manual entity tracking needed - memory handles it

**Key insight:** Memory-based grounding is what makes agents feel intelligent and context-aware. Without it, every reference needs to be explicit, making conversations robotic and frustrating.